In [5]:
using ForwardDiff, LinearAlgebra

# Gaussian kernel
k(x₁, x₂, θ₁, θ₂, θ₃) = θ₁*exp(-0.5*inv(θ₂^2)*(x₁ - x₂)^2) + θ₃*(x₁==x₂)

# Gradient function
∂k∂θ₁(x₁, x₂, θ₁, θ₂, θ₃) = ForwardDiff.derivative(θ₁ -> k.(x₁, x₂, θ₁, θ₂, θ₃), θ₁)
∂k∂θ₂(x₁, x₂, θ₁, θ₂, θ₃) = ForwardDiff.derivative(θ₂ -> k.(x₁, x₂, θ₁, θ₂, θ₃), θ₂)
∂k∂θ₃(x₁, x₂, θ₁, θ₂, θ₃) = ForwardDiff.derivative(θ₃ -> k.(x₁, x₂, θ₁, θ₂, θ₃), θ₃)
∂L∂θ(K, ∂K∂θ, Yᵗ) = -tr(inv(K)*∂K∂θ) + (inv(K)*Yᵗ)'*∂K∂θ*(inv(K)*Yᵗ)

# Gaussian process
function gp_reg(Xᵗ, Yᵗ, Xᵖ, θ₁, θ₂, θ₃)
    # Training
    Kᵗᵗ = k.(Xᵗ, Xᵗ', θ₁, θ₂, θ₃)
    invKᵗᵗ = inv(Kᵗᵗ)
    likelihood = -log(det(Kᵗᵗ)).- dot(Yᵗ', invKᵗᵗ, Yᵗ) 
    
    # Prediction
    Kᵖᵖ, Kᵖᵗ = (k.(Xᵖ, Xᵖ', θ₁, θ₂, θ₃), k.(Xᵖ, Xᵗ', θ₁, θ₂, θ₃))
    μᵖ, Σᵖ = (Kᵖᵗ*invKᵗᵗ*Yᵗ, Kᵖᵖ - Kᵖᵗ*invKᵗᵗ*Kᵖᵗ')
    
    # Gradient
    ∂K∂θ₁ = ∂k∂θ₁(Xᵗ, Xᵗ', θ₁, θ₂, θ₃)
    ∂K∂θ₂ = ∂k∂θ₂(Xᵗ, Xᵗ', θ₁, θ₂, θ₃)
    ∂K∂θ₃ = ∂k∂θ₃(Xᵗ, Xᵗ', θ₁, θ₂, θ₃)
    
    ∂L∂θ₁ = ∂L∂θ(Kᵗᵗ, ∂K∂θ₁, Yᵗ)
    ∂L∂θ₂ = ∂L∂θ(Kᵗᵗ, ∂K∂θ₂, Yᵗ)
    ∂L∂θ₃ = ∂L∂θ(Kᵗᵗ, ∂K∂θ₃, Yᵗ)
    
    likelihood, μᵖ, Σᵖ, ∂K∂θ₁, ∂K∂θ₂, ∂K∂θ₃, ∂L∂θ₁, ∂L∂θ₂, ∂L∂θ₃ 
end

# Train dataset
Xᵗ = range(-5, 5, length=10)
Yᵗ = sin.(2pi*Xᵗ ./10) + 0.1*randn(10)

# Test dataset
Xᵖ = range(-8, 8, length=100)

# Parameters
θ₁, θ₂, θ₃ = (1.0, 1.0, 1.0)
γ = 0.05

for i in 1:10
    likelihood, μᵖ, Σᵖ, ∂K∂θ₁, ∂K∂θ₂, ∂K∂θ₃, ∂L∂θ₁, ∂L∂θ₂, ∂L∂θ₃ = gp_reg(Xᵗ, Yᵗ, Xᵖ, θ₁, θ₂, θ₃)
    θ₁ += γ * ∂L∂θ₁
    θ₂ += γ * ∂L∂θ₂
    θ₃ += γ * ∂L∂θ₃
    println("Epoch: $i, likelihood : $likelihood, θ₁ :$θ₁, θ₂ : $θ₂, θ₃: $θ₃")
end

Epoch: 1, likelihood : -7.868949723982096, θ₁ :0.8366278297084455, θ₂ : 1.1118775389573887, θ₃: 0.7446620802970594
Epoch: 2, likelihood : -5.5615061355164235, θ₁ :0.6562263079928039, θ₂ : 1.2261961886959183, θ₃: 0.40809361107045666
Epoch: 3, likelihood : -1.71379335658488, θ₁ :0.44586797850487503, θ₂ : 1.3640998646766156, θ₃: -0.1399375651996248
Epoch: 4, likelihood : 7.532625659644252, θ₁ :1.7007569724347285, θ₂ : 0.19375297306144978, θ₃: 4.717137459632823
Epoch: 5, likelihood : -19.28276378946086, θ₁ :1.6282399094464037, θ₂ : 0.1937531356502066, θ₃: 4.644620396081278
Epoch: 6, likelihood : -19.070183734987616, θ₁ :1.5541736928029979, θ₂ : 0.19375329859160453, θ₃: 4.5705541788482895
Epoch: 7, likelihood : -18.848322860522384, θ₁ :1.478455876921499, θ₂ : 0.19375346173937494, θ₃: 4.494836362348328
Epoch: 8, likelihood : -18.61634176148586, θ₁ :1.4009723033580843, θ₂ : 0.19375362491186826, θ₃: 4.417352788134675
Epoch: 9, likelihood : -18.37328259619557, θ₁ :1.321595149696241, θ₂ : 0.1937